In [1]:
const dfd = require("danfojs-node");
const tf = dfd.tensorflow
// const tf = require('@tensorflow/tfjs');

In [2]:
config = {
  rowStart: 7228,
  rowEnd: 7288,
  timesteps: 2,
  maxBatchSize: 13,
  layerSize: 1,
  unitSize: 1,
  dropout: 0,
  learningRate: 0.1,
  maxEpochs: 1,
  timeCol: 'tanggal',
  feature: ['rr']
}

// usecols = [config.timeCol].concat(config.feature)

function trainTestSplit(dataset, timeStep = 1) {
  dataX = []
  dataY = []

  // for (let i = 0; i < dataset.values.length - timeStep - 1; i++) {
  for (let i = 0; i < dataset.values.length - timeStep; i++) {
    dataX.push(dataset.iloc({ rows: [`${i}:${i + timeStep}`] }).values)
    dataY.push(dataset.iloc({ rows: [i + timeStep] }).values[0])
  }

  return [dataX, dataY]
}

{
  rowStart: 7228,
  rowEnd: 7288,
  timesteps: 2,
  maxBatchSize: 13,
  layerSize: 1,
  unitSize: 1,
  dropout: 0,
  learningRate: 0.1,
  maxEpochs: 1,
  timeCol: 'tanggal',
  feature: [ 'rr' ]
}

In [3]:
$$async$$ = true;
(async function () {
  dataset = await dfd.readCSV("../Data/1985-2021.csv")
  dataset = dataset.iloc({rows:[`${config.rowStart}:${config.rowEnd}`]})
  datelist = dataset.loc({columns: [config.timeCol]})

  $$done$$(dataset.shape)
})()

[ 60, 11 ]

In [4]:
featureset = dataset.loc({columns: config.feature})

scaller = new dfd.MinMaxScaler()
scaller.fit(featureset)

featuresetScaled = scaller.transform(featureset)

trainSize = featuresetScaled.values.length * 0.9
trainset = featuresetScaled.iloc({rows: [`${0}:${trainSize}`]})
testset =  featuresetScaled.iloc({rows: [`${trainSize}:${featuresetScaled.values.length}`]})

trainset = trainTestSplit(trainset, timeStep=config.timesteps)
testset = trainTestSplit(testset, timeStep=config.timesteps)

XTrain = tf.tensor(trainset[0])
yTrain = tf.tensor(trainset[1])

XTest = tf.tensor(testset[0])
yTest = tf.tensor(testset[1])

XTrain = tf.reshape(XTrain, [XTrain.shape[0], config.feature.length, XTrain.shape[1]])
XTest = tf.reshape(XTest, [XTest.shape[0], config.feature.length, XTest.shape[1]])

// XTrain = tf.reshape(XTrain, [4,13, 1, 2])
// yTrain = tf.reshape(yTrain, [4,13, 1])

Tensor {
  kept: false,
  isDisposedInternal: false,
  shape: [ 4, 1, 2 ],
  dtype: 'float32',
  size: 8,
  strides: [ 2, 2 ],
  dataId: {},
  id: 18,
  rankType: '3',
  scopeId: 6
}

In [5]:
XTrain.shape

[ 52, 1, 2 ]

In [6]:
yTrain.shape

[ 52, 1 ]

In [24]:
yPredLoss = []
yTrueLoss = []

model = tf.sequential()

model.add(tf.layers.lstm({
  units: config.unitSize,
  batchSize: config.maxBatchSize,
  inputShape: [config.feature.length, config.timesteps],
  goBackwards: true,
  dropout: config.dropout,
  weights: [
    tf.tensor([
      [0.5774, 0.5774, 0.5774, 0.5774],
      [0.5774, 0.5774, 0.5774, 0.5774]
    ]),
    tf.tensor([
      [0.5774, 0.5774, 0.5774, 0.5774]
    ]),
    tf.zeros([4])
  ]
}))

// model.layers[0].getWeights()[0].print()

model.compile({
  optimizer: tf.train.GradientDescentOptimizer(config.learningRate),
  // loss: 'meanSquaredError',
  loss: function (yTrue, yPred) {
    yPredLoss.push(yPred.array())
    yTrueLoss.push(yTrue.array())
    return tf.losses.meanSquaredError(yTrue, yPred)
  },
  runEagerly: true
})

model.summary()

TypeError: tf.train.gdo is not a function

In [19]:
batchLoss = []

$$async$$ = true;
(async function () {
  history = await model.fit(XTrain, yTrain, {
    shuffle: false,
    epochs: config.maxEpochs,
    verbose: 1,
    batchSize: config.maxBatchSize,
    callbacks: [
      new tf.CustomCallback({
        onBatchEnd: function (batch, logs = null) {
          batchLoss.push(logs)
        }
      })
    ]
  })
  $$done$$();
})()

Epoch 1 / 1


220ms 4232us/step - loss=0.0824 


In [20]:
batchLoss

[
  { batch: 0, size: 13, loss: 0.11157790571451187 },
  { batch: 1, size: 13, loss: 0.04249683767557144 },
  { batch: 2, size: 13, loss: 0.08604662120342255 },
  { batch: 3, size: 13, loss: 0.08932735025882721 }
]

In [22]:
yPredLoss[0]

Promise {
  [
    [ 0.003764435416087508 ],
    [ 0.19518616795539856 ],
    [ 0.3006698787212372 ],
    [ 0.2219531536102295 ],
    [ 0.16048069298267365 ],
    [ 0.05620086193084717 ],
    [ 0.04501638934016228 ],
    [ 0.030724365264177322 ],
    [ 0.0031562268268316984 ],
    [ 0.024033837020397186 ],
    [ 0.03903147950768471 ],
    [ 0.061161961406469345 ],
    [ 0.07305235415697098 ]
  ]
}

In [21]:
yTrueLoss[0]

Promise {
  [
    [ 1 ],
    [ 0.5257731676101685 ],
    [ 0.6206185817718506 ],
    [ 0.23711340129375458 ],
    [ 0.10309278219938278 ],
    [ 0.17525772750377655 ],
    [ 0.020618556067347527 ],
    [ 0.0010309278732165694 ],
    [ 0.15463916957378387 ],
    [ 0.08969072252511978 ],
    [ 0.2773195803165436 ],
    [ 0.15257732570171356 ],
    [ 0.4402061998844147 ]
  ]
}